# Load the libraries

In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import time

import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from pathlib import Path

from tqdm import tqdm
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import random
import gc
import cv2

import tensorflow as tf
# tf.get_logger().setLevel('ERROR')
# tf.autograph.set_verbosity(2)
from tensorflow.keras import models, layers, optimizers, applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_zh9lekq because the default path (/home/macot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Use OpenCV to load webcam/video

## Load the pretrained model

The best model was VGG16, and this was used to predict the characters

In [2]:
model = tf.keras.models.load_model('vgg16_v2.h5')

2022-03-13 14:35:33.918574: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-13 14:35:33.918713: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 64, 64, 3)         0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 19)                38931     
                                                                 
Total params: 14,753,619
Trainable params: 14,753,619
Non-trainable params: 0
_________________________________________________________________


In [4]:
labels = ['a', 'ba', 'dara', 'ei', 'ga', 'ha', 'ka', 'kuw', 'la',
          'ma', 'na', 'nga', 'ou', 'pa', 'sa', 'ta', 'tul', 'wa', 'ya']
label_dict = {i: j for i, j in zip(range(19), sorted(labels))}
label_dict

{0: 'a',
 1: 'ba',
 2: 'dara',
 3: 'ei',
 4: 'ga',
 5: 'ha',
 6: 'ka',
 7: 'kuw',
 8: 'la',
 9: 'ma',
 10: 'na',
 11: 'nga',
 12: 'ou',
 13: 'pa',
 14: 'sa',
 15: 'ta',
 16: 'tul',
 17: 'wa',
 18: 'ya'}

# Predict from Video
enter the filepath of the video file in the code `cap = cv2.VideoCapture(filepath)` as a string.

In [ ]:
## input the filename here
cap = cv2.VideoCapture('baybayin_2.mov')

if not cap.isOpened():
    print("Cannot open camera")
    exit()

width = 640
height = 480
cap.set(3, width)
cap.set(4, height)

time_start = time.time()

## THIS WAS ADDED TO TRY TO SAVE THE VIDEO
writer = cv2.VideoWriter('demo', 
                         cv2.VideoWriter_fourcc(*'XVID'),25, (width, height))


while True:
    try:
        success, imgOriginal = cap.read()
        img = cv2.resize(imgOriginal, (64, 64))
        img = np.expand_dims(img, axis=0)
        
        # Write the video
        writer.write(imgOriginal) ## THIS WAS ADDED TO SAVE THE VID

        #predict
        predictions = model.predict(img)
        pred_label = label_dict[np.argmax(predictions)]
        probVal = np.amax(predictions)
        
        threshold = 0.80
        if probVal > threshold:
            cv2.putText(img=imgOriginal, text=pred_label + ': ' + str(probVal),
                        org=(100, 100), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                        fontScale=3, color=(255,255,0), thickness=3)
            print(pred_label, probVal)
        else:
            cv2.putText(img=imgOriginal, text='Unable to predict',
                        org=(100, 100), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                        fontScale=3, color=(255,255,0), thickness=3)
            print(pred_label, probVal)

        cv2.imshow('BaybAIyin Predictor', imgOriginal)
        
        time_now = time.time()
        num_seconds = time_now - time_start
        if num_seconds > 90:
            print('Time is up.')
            break

    except:
        print('Error: Closing camera')
        success = False
        cap.release()
        cv2.destroyAllWindows()
        break

    if (cv2.waitKey(1) & 0xFF == ord('q')):
        print('Camera closed by user')
        success = False
        cap.release()
        cv2.destroyAllWindows()
        break

    if not success:
        print('Failed to capture image')
        cap.release()
        cv2.destroyAllWindows()
        break

cap.release()
writer.release() ## THIS WAS ADDED
cv2.destroyAllWindows()
print('Video End')


systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



[ERROR:0@11.522] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap.cpp (597) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.5) /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): demo in function 'icvExtractPattern'


2022-03-13 14:35:34.557668: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-13 14:35:34.615406: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


sa 0.9832792
sa 0.9866077
sa 0.9811701
sa 0.9872661
sa 0.9587432
sa 0.9334822
sa 0.9530434
sa 0.9583482
sa 0.9683574
sa 0.9588049
sa 0.94433373
sa 0.9463751
sa 0.9734914
sa 0.98681647
sa 0.98800474
sa 0.9840882
sa 0.975453
sa 0.9266179
sa 0.9206156
sa 0.942889
sa 0.9757287
sa 0.9601366
sa 0.9190487
sa 0.8969401
sa 0.9805833
sa 0.95540184
sa 0.9664624
sa 0.889827
sa 0.84830415
sa 0.85565484
sa 0.9507069
sa 0.99113876
sa 0.9891131
sa 0.9749289
sa 0.89930844
sa 0.7138174
na 0.40029827


# Predict from camera Feed

In [ ]:
# # Put 0 to acces your webcam
# cap = cv2.VideoCapture(0)

# if not cap.isOpened():
#     print("Cannot open camera")
#     exit()

# width = 640
# height = 480
# cap.set(3, width)
# cap.set(4, height)

# time_start = time.time()


# while True:
#     try:
#         success, imgOriginal = cap.read()
#         img = cv2.resize(imgOriginal, (64, 64))
#         img = np.expand_dims(img, axis=0)

#         #predict
#         predictions = model.predict(img)
#         pred_label = label_dict[np.argmax(predictions)]
#         probVal = np.amax(predictions)
        
#         threshold = 0.80
#         if probVal > threshold:
#             cv2.putText(img=imgOriginal, text=pred_label + ': ' + str(probVal),
#                         org=(100, 100), fontFace=cv2.FONT_HERSHEY_COMPLEX,
#                         fontScale=3, color=(255,255,0), thickness=3)
#             print(pred_label, probVal)
#         else:
#             cv2.putText(img=imgOriginal, text='Unable to predict',
#                         org=(100, 100), fontFace=cv2.FONT_HERSHEY_COMPLEX,
#                         fontScale=3, color=(255,255,0), thickness=3)
#             print(pred_label, probVal)

#         cv2.imshow('BaybAIyin Prediction', imgOriginal)
        
#         time_now = time.time()
#         num_seconds = time_now - time_start
#         if num_seconds > 90:
#             print('Time is up.')
#             break

#     except:
#         print('Error: Closing camera')
#         success = False
#         cap.release()
#         cv2.destroyAllWindows()
#         break

#     if (cv2.waitKey(1) & 0xFF == ord('q')):
#         print('Camera closed by user')
#         success = False
#         cap.release()
#         cv2.destroyAllWindows()
#         break

#     if not success:
#         print('Failed to capture image')
#         cap.release()
#         cv2.destroyAllWindows()
#         break

# cap.release()
# cv2.destroyAllWindows()
# print('Camera closed')

In [ ]:
from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="baybayin2_demo.mp4" type="video/mp4">
</video>
""")